In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [2]:
class BPR(nn.Module):
    def __init__(self):
        super(BPR, self).__init__()
        self.W = None             # user matrix
        self.H = None             # item matrix
        self.Wsc = None           # scorer
        self.Hsc = None 
        
        self.uid = None
        self.iid = None
        
        self.user_items = {}
        self.dev_user_items = {}
        
        self.rating_exp = None   # softmax sum
        self.rating_exp_mul_H = None
    
    def preprocess(self, df, train_size=0.8, test_size=0.1):
        df = df.rename(columns = {df.columns[0]: 'ori_uid', df.columns[1]: 'ori_iid', df.columns[2]: 'rating'})
        
        uid_map = pd.DataFrame({"ori_uid": np.asarray(list(set(df.iloc[:,0].values)))})
        uid_map["serial_uid"] = uid_map.index
        iid_map = pd.DataFrame({"ori_iid": np.asarray(list(set(df.iloc[:,1].values)))})
        iid_map["serial_iid"] = iid_map.index
        
        self.uid = uid_map["serial_uid"].values
        self.iid = iid_map["serial_iid"].values
        
        df = df.merge(uid_map, left_on = 'ori_uid', right_on = 'ori_uid', how="left")
        df = df.merge(iid_map, left_on = 'ori_iid', right_on = 'ori_iid', how="left")
        df = df[['serial_uid', 'serial_iid', 'rating']]
        
        train, test = self._split(df, train_size)
        test, dev = self._split(test, test_size / (1 - train_size))
        return train, test, dev
    
    def _split(self, df, ratio):
        train = pd.DataFrame(columns = df.columns, dtype=int)
        test = pd.DataFrame(columns = df.columns, dtype=int)
        for i in self.uid:
            train_1, test_1 = train_test_split(df[df.iloc[:, 0] == i], train_size = ratio, shuffle = True, random_state = 5)
            train = pd.concat([train, train_1])
            test = pd.concat([test, test_1])
        return train, test
    
    def generate_train_batch(self, batch, sets):
        train = []
        for b in range(batch):
            u = self.uid[np.random.randint(0, len(self.uid))]
            i = sets[u][np.random.randint(0, len(sets[u]))]
            j = self.iid[np.random.randint(0, len(self.iid))]
            while j in sets[u]:
                j = self.iid[np.random.randint(0, len(self.iid))]
            train.append([u, i, j])
        return np.asarray(train) 

    def forward(self, uids, iids, device):
        self.rating_exp = torch.zeros(len(self.uid)).to(device)
        self.rating_exp_mul_H = torch.zeros([len(self.uid), self.H.shape[1]]).to(device)
        
        # 处理 idx 得到 embedded Wu Hi
        emb_idxs = [self.user_items[uid] for uid in uids]
        item_emb = nn.utils.rnn.pad_sequence([self.Hsc[emb_idx] for emb_idx in emb_idxs], batch_first=True)
        user_emb = self.Wsc[uids][:, None, :]
        
        # 计算批次内 user_item 得分
        user_item_exp_sc = torch.sum(item_emb * user_emb, dim = -1)
        mask = (user_item_exp_sc != 0).type(torch.float32)
        # 取指数， mask 保证补 0 位还是 0
        user_item_exp_sc = torch.exp(user_item_exp_sc) * mask
        
        # 计算指数和
        self.rating_exp_mul_H[uids] = torch.sum(user_item_exp_sc.unsqueeze(2).repeat(1, 1, self.H.shape[1]) * item_emb, dim = 1)
        self.rating_exp[uids] = torch.sum(user_item_exp_sc, dim = 1)
        #返回 softmax probablilty of item i among user_items
        return torch.exp(torch.sum(self.Wsc[uids] * self.Hsc[iids], dim = 1)) / self.rating_exp[uids]
            
    def fit_dds(self, df, dev, k, stepsize=0.05, max_iter=10, batch=10000):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        # 初始化 W，H
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)   
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01) 
        
        # 初始化 scorer
        self.Wsc = torch.rand(len(self.uid), k).to(device) * 0.01    
        self.Hsc = torch.rand(len(self.iid), k).to(device) * 0.01  
        
        # 创建字典：用户u对应他访问过的所有items集合    
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        self.dev_user_items = dev.groupby(dev.columns[0])[dev.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        # 主模型优化器        
        optimizer = optim.SGD([self.W, self.H], lr = 1)
        
        for x in range(max_iter):            
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 评分器概率分布，forward 返回 softmax 概率分布
            score_prob = self.forward(u, i, device)     
            
            # 主模型参数更新
            optimizer.zero_grad() 
            score_loss = -torch.sum(score_prob * torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb-j_emb),dim = 1))))
            bpr_loss = -torch.mean(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            score_loss.backward()
            optimizer.step()

            # 训练集上 W,H 的梯度
            W_grad_sum = self.W.grad.clone()
            H_grad_sum = self.H.grad.clone()
            
            # 对数概率分布下 Wsc, Hsc 梯度
            log_prob_Wsc_grad = torch.zeros((len(self.uid), k)).to(device)
            log_prob_Hsc_grad = torch.zeros((len(self.iid), k)).to(device)
            log_prob_Wsc_grad[u] = self.Hsc[i] - self.rating_exp_mul_H[u] / self.rating_exp[u].unsqueeze(1).repeat(1, k)
            log_prob_Hsc_grad[i] = self.Wsc[u] * ((1 - score_prob).unsqueeze(1).repeat(1, k))
            
            #取 dev uij三元组
            uij = self.generate_train_batch(5000, self.dev_user_items)
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            # 计算 dev 集上 W,H 的梯度
            optimizer.zero_grad()
            dev_loss = -torch.sum(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            dev_loss.backward()
            W_grad_dev_sum = self.W.grad.clone()    
            H_grad_dev_sum = self.H.grad.clone()

            # 计算 reward: reward 为 W,H 在训练集和 dev 集上的梯度积    
            r_W = torch.sum(W_grad_sum * W_grad_dev_sum, dim=1)
            r_H = torch.sum(H_grad_sum * H_grad_dev_sum, dim=1)
            r_W = r_W.unsqueeze(1).repeat(1, k)
            r_H = r_H.unsqueeze(1).repeat(1, k)

            # Wsc，Hsc 更新
            self.Wsc += r_W * log_prob_Wsc_grad
            self.Hsc += r_H * log_prob_Hsc_grad

            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {bpr_loss.item()}")
    
    def fit_ori(self, df, k, stepsize=0.05, max_iter=10, batch=10000):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.to(device)
        
        self.W = nn.Parameter(torch.rand(len(self.uid), k).to(device) * 0.01)    # 初始化 W，H
        self.H = nn.Parameter(torch.rand(len(self.iid), k).to(device) * 0.01)  
        
        # 创建字典：用户u对应他访问过的所有items集合
        self.user_items = df.groupby(df.columns[0])[df.columns[1]].apply(lambda x: np.array(x)).to_dict()
        
        optimizer = optim.SGD([self.W, self.H], lr = stepsize)     # 主模型优化器
        for x in range(max_iter):
            #取训练批次：uij三元组
            uij = self.generate_train_batch(batch, self.user_items)
            
            u = uij[:, 0]
            i = uij[:, 1]
            j = uij[:, 2]
            u_emb = self.W[u]
            i_emb = self.H[i]
            j_emb = self.H[j]
            
            optimizer.zero_grad()
            loss = -torch.sum(torch.log(torch.sigmoid(torch.sum(u_emb * (i_emb - j_emb),dim = 1))))
            loss.backward()
            optimizer.step()
            
            if ( x + 1 ) % 10 == 0:
                print(f"Iteration: {x+1}, BPR loss: {loss.item() / batch}")
    
    def _predict(self, uid, items, n):
        scores = torch.mv(self.H[items], self.W[uid])
        top_N = list(zip(items, scores.detach().cpu().numpy()))
        return sorted(top_N, key=lambda s: s[1], reverse=True)[:n]

    def NDCG(self, uid, test, n):         # 用模型排序+真实分数计算 DCG, 重排后计算 iDCG
        test_user = test[test.iloc[:, 0] == uid]
        rating = self._predict(uid, test_user.iloc[:, 1].values, n)
        irating =sorted(test_user.iloc[:, 2].values, reverse=True)
        dcg = 0
        idcg = 0
        if n > len(irating): n = len(irating)  
        for i in range(n):
            r = test_user[test_user.iloc[:, 1]==rating[i][0]].iloc[0, 2]
            dcg += 1.0 * (2**r - 1) / math.log(i + 2, 2)
            idcg += 1.0 * (2**irating[i] - 1) / math.log(i + 2, 2)
        return dcg / idcg

    def performance(self, test, n):      # Output recall@n, precision@n, NDCG@n
        hit = 0
        n_recall = 0
        n_precision = 0
        ndcg = 0
        for i in self.uid:
            # Items that User i hasn't tried in training set
            unknown_items = np.setdiff1d(self.iid, self.user_items[i])
            # Items that User i actually tried in testing set
            known_items = test[test.iloc[:, 0] == i].iloc[:, 1].values
            
            #目标：预测 unknown items 中的top_N，若击中test中的items，则为有效预测
            ru = self._predict(i, unknown_items, n)
            for item, pui in ru:
                if item in known_items:
                    hit += 1
            n_recall += len(known_items)
            n_precision += n
            ndcg += self.NDCG(i, test, n)
            
        recall = hit / (1.0 * n_recall)
        precision = hit / (1.0 * n_precision)
        ndcg /= len(self.uid)
        return recall, precision, ndcg

In [3]:
df1 = pd.read_csv("./ml-100k/u.data", sep="\t", names=['user id', 'item id', 'rating', 'timestamp'])
df2 = pd.read_csv("./ml-1m/ratings.dat", sep="::", names=['user id', 'item id', 'rating', 'timestamp'], engine='python')

### 100K

In [6]:
model1 = BPR()
model2 = BPR()
train1, test1, dev1 = model1.preprocess(df1)
train2, test2, dev2 = model2.preprocess(df1)
print(train1.shape)
print(test1.shape)
print(dev1.shape)

(79619, 3)
(9942, 3)
(10439, 3)


### 100K Pure BPR

In [13]:
%%time
model1.fit_ori(train1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.62378154296875
Iteration: 20, BPR loss: 0.35774130859375
Iteration: 30, BPR loss: 0.3063040283203125
Iteration: 40, BPR loss: 0.305498193359375
Iteration: 50, BPR loss: 0.29411689453125
Iteration: 60, BPR loss: 0.2744450439453125
Iteration: 70, BPR loss: 0.2479959716796875
Iteration: 80, BPR loss: 0.214836279296875
Iteration: 90, BPR loss: 0.2020580810546875
Iteration: 100, BPR loss: 0.1797273193359375
Iteration: 110, BPR loss: 0.16960958251953126
Iteration: 120, BPR loss: 0.169327880859375
Iteration: 130, BPR loss: 0.16193299560546875
Iteration: 140, BPR loss: 0.15832347412109374
Iteration: 150, BPR loss: 0.1492018310546875
Iteration: 160, BPR loss: 0.1437028564453125
Iteration: 170, BPR loss: 0.1342430908203125
Iteration: 180, BPR loss: 0.13368096923828124
Iteration: 190, BPR loss: 0.1286603515625
Iteration: 200, BPR loss: 0.1243393798828125
Iteration: 210, BPR loss: 0.11574803466796875
Iteration: 220, BPR loss: 0.1156223388671875
Iteration: 230, BPR loss: 

In [14]:
%%time
n = 10
rec, pre, ndcg = model1.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.13870625662778366
Recall@10: 0.1315630657815329
NDCG@10: 0.8245452801685419
CPU times: total: 3.03 s
Wall time: 8.4 s


### 100K BPR + Data Selection

In [7]:
%%time
model2.fit_dds(train1, dev1, k = 50, max_iter = 500)

Iteration: 10, BPR loss: 0.665634274482727
Iteration: 20, BPR loss: 0.5026760101318359
Iteration: 30, BPR loss: 0.4225527346134186
Iteration: 40, BPR loss: 0.3804778754711151
Iteration: 50, BPR loss: 0.35433879494667053
Iteration: 60, BPR loss: 0.33763349056243896
Iteration: 70, BPR loss: 0.32537055015563965
Iteration: 80, BPR loss: 0.3282545506954193
Iteration: 90, BPR loss: 0.30993345379829407
Iteration: 100, BPR loss: 0.3006282448768616
Iteration: 110, BPR loss: 0.2878747582435608
Iteration: 120, BPR loss: 0.2775033116340637
Iteration: 130, BPR loss: 0.2510831356048584
Iteration: 140, BPR loss: 0.2414250671863556
Iteration: 150, BPR loss: 0.24074730277061462
Iteration: 160, BPR loss: 0.21577690541744232
Iteration: 170, BPR loss: 0.21319936215877533
Iteration: 180, BPR loss: 0.2050953358411789
Iteration: 190, BPR loss: 0.19327235221862793
Iteration: 200, BPR loss: 0.18959447741508484
Iteration: 210, BPR loss: 0.186885803937912
Iteration: 220, BPR loss: 0.17993134260177612
Iteration: 

In [8]:
%%time
n = 10
rec, pre, ndcg = model2.performance(test1, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.15906680805938495
Recall@10: 0.15087507543753773
NDCG@10: 0.8299234526856015
CPU times: total: 219 ms
Wall time: 2.7 s


### 1M

In [38]:
model3 = BPR()
model4 = BPR()
train3, test3, dev3 = model3.preprocess(df2)
train4, test4, dev4 = model4.preprocess(df2)
print(train3.shape)
print(test3.shape)
print(dev3.shape)

(797758, 3)
(99692, 3)
(102759, 3)


### 1M Pure BPR

In [39]:
%%time
model3.fit_ori(train3, k = 20, max_iter = 500)

Iteration: 10, BPR loss: 0.3754083514213562
Iteration: 20, BPR loss: 0.381619393825531
Iteration: 30, BPR loss: 0.3569476902484894
Iteration: 40, BPR loss: 0.34104621410369873
Iteration: 50, BPR loss: 0.32315343618392944
Iteration: 60, BPR loss: 0.31233617663383484
Iteration: 70, BPR loss: 0.2830395996570587
Iteration: 80, BPR loss: 0.27295058965682983
Iteration: 90, BPR loss: 0.25953519344329834
Iteration: 100, BPR loss: 0.2350631058216095
Iteration: 110, BPR loss: 0.2118961364030838
Iteration: 120, BPR loss: 0.2172245979309082
Iteration: 130, BPR loss: 0.21276317536830902
Iteration: 140, BPR loss: 0.20478075742721558
Iteration: 150, BPR loss: 0.19854378700256348
Iteration: 160, BPR loss: 0.2073565423488617
Iteration: 170, BPR loss: 0.20717592537403107
Iteration: 180, BPR loss: 0.18923752009868622
Iteration: 190, BPR loss: 0.1915675401687622
Iteration: 200, BPR loss: 0.18511264026165009
Iteration: 210, BPR loss: 0.1853107064962387
Iteration: 220, BPR loss: 0.18176881968975067
Iteratio

In [40]:
%%time
n = 10
rec, pre, ndcg = model3.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.10697019867549669
Recall@10: 0.06480961360991855
NDCG@10: 0.7990015927015258
CPU times: total: 6.41 s
Wall time: 17.8 s


### 1M BPR + Data Selection

In [41]:
%%time
model4.fit_dds(train3, dev3, k = 20, max_iter = 500)

Iteration: 10, BPR loss: 0.410556823015213
Iteration: 20, BPR loss: 0.3687320053577423
Iteration: 30, BPR loss: 0.3714205324649811
Iteration: 40, BPR loss: 0.35557058453559875
Iteration: 50, BPR loss: 0.3410336673259735
Iteration: 60, BPR loss: 0.32621532678604126
Iteration: 70, BPR loss: 0.31507599353790283
Iteration: 80, BPR loss: 0.2985517978668213
Iteration: 90, BPR loss: 0.2722259759902954
Iteration: 100, BPR loss: 0.26787757873535156
Iteration: 110, BPR loss: 0.24591214954853058
Iteration: 120, BPR loss: 0.24201327562332153
Iteration: 130, BPR loss: 0.2360617071390152
Iteration: 140, BPR loss: 0.22995702922344208
Iteration: 150, BPR loss: 0.20491468906402588
Iteration: 160, BPR loss: 0.22022663056850433
Iteration: 170, BPR loss: 0.20099157094955444
Iteration: 180, BPR loss: 0.21964798867702484
Iteration: 190, BPR loss: 0.20061607658863068
Iteration: 200, BPR loss: 0.20826725661754608
Iteration: 210, BPR loss: 0.2060583382844925
Iteration: 220, BPR loss: 0.2017163783311844
Iterati

In [42]:
%%time
n = 10
rec, pre, ndcg = model4.performance(test3, n)
print(f'Precision@{n}: {pre}')
print(f'Recall@{n}: {rec}')
print(f'NDCG@{n}: {ndcg}')

Precision@10: 0.10029801324503311
Recall@10: 0.06076716286161377
NDCG@10: 0.8010941744826243
CPU times: total: 5.92 s
Wall time: 18.9 s
